In [ ]:
%%writefile FileManager.h
#ifndef FILEMANAGER_H
#define FILEMANAGER_H

#include <iostream>
#include "Student_manager.h"
#include "FriendsManager.h"
#include "MessageManager.h"
using namespace std;

// The FileManager class is responsible for centralized loading and saving of application data.
// It manages persistence (read/write to files) and provides access to core managers.

class FileManager {
  private:
    string profFileName;
    string credFileName;
    string frndFileName;

  public:
    // Constructor
    FileManager();

    // Reads all necessary data (students, friends, messages, etc.) from files
    void loadAll(Student_manager&, FriendsManager&, MessageManager&);

    // Saves all necessary data to files
    void saveAll(Student_manager&, FriendsManager&, MessageManager&);
};

#endif // FILEMANAGER_H

In [ ]:
%%writefile FileManager.cpp

#include "FileManager.h"
#include <string>
#include <fstream>

FileManager::FileManager() {
  profFileName = "profiles.txt";
  credFileName = "credentials.txt";
  frndFileName = "friends.txt";
}

// Load Student Profiles and Credentials
void FileManager::loadAll(Student_manager& sm, FriendsManager& fm, MessageManager& mm){
  try {
    // --- Load Student Profiles and Credentials ---
    // open profiles file
    ifstream studfile(profFileName);
    if (!studfile.is_open()) {
      string exceptionString = "Failed to open " + profFileName + " file.";
      throw exceptionString;
    }

    string matricNo, name, email, course, password;

    // load profiles into sm
    while (getline(studfile, matricNo, ',') &&
            getline(studfile, name, ',') &&
            getline(studfile, email, ',') &&
            getline(studfile, course)) {

      Student* newStudent = new Student(matricNo, name, email, course); // create new Student object
      sm.sortedInsert(newStudent);  // store Student data into the linked-list
    }
    studfile.close(); //close file

    // open credentials file
    ifstream credfile(credFileName);
    if (!credfile.is_open()) {
      string exceptionString = "Failed to open " + credFileName + " file.";
      throw exceptionString;
    }
    // load credentials into sm
    while (getline(credfile, email, ',') && getline(credfile, password)) {
      Node* stud = sm.searchData(EMAIL, email); // search for the Student in the student list with the email
      if (stud != nullptr) { // if Student is found
        stud->data->setPassword(password); // store password into the Student node
      } else{
        string exceptionString = "Student credentials not found for email: " + email;
        throw exceptionString;
      }
    }
    credfile.close();

    // --- Load Friends ---
    ifstream friendFile(frndFileName);
    if (!friendFile.is_open()) {
      string exceptionString = "Failed to open " + frndFileName + " file.";
      throw exceptionString;
    }
    string id1, id2;
    while (getline(friendFile, id1, ',') && getline(friendFile, id2)) {
      fm->addFriendship(id1, id2); // You must implement this to add both ways in memory
    }
    friendFile.close();

    // --- Load Messages ---
    // For each student, open messages_<matricNo>.txt and load into mm
    Node* current = sm->getHead();
    while (current) {
      string filename = "messages_" + current->data->getMatricNo() + ".txt";
      ifstream msgfile(filename.c_str());

      if (!msgfile.is_open()) {
        string exceptionString = "Failed to open " + filename + " file.";
        throw exceptionString;
      }
      string line;

      while (getline(msgfile, line)) {
        mm->loadMessageLine(current->data->getMatricNo(), line);
      }
      msgfile.close();
      current = current->next;
    }
  }
  catch (string errMsg) {
      cout << "FileManager Error: " << errMsg << endl;
  }
  catch (...) {
      cout << "FileManager Error: Unknown exception occurred during file loading." << endl;
  }
}

// Save all data from memory to files
void FileManager::saveAll(Student_manager& sm, FriendsManager& fm, MessageManager& mm) {
  // --- Save Student Profiles ---
  ofstream profiles(profFileName);
  Node* current = sm->getHead();
  while (current) {
    Student* s = current->data;
    profiles << s->getMatricNo() << "," << s->getFullName() << "," << s->getEmail() << "," << s->getCourseName() << std::endl;
    current = current->next;
  }
  profiles.close();

  // --- Save Credentials ---
  ofstream credentials(credFileName);
  current = sm->getHead();
  while (current) {
    Student* s = current->data;
    credentials << s->getEmail() << "," << s->getPassword() << std::endl;
    current = current->next;
  }
  credentials.close();

  // --- Save Friends ---
  ofstream friends(frndFileName);
  fm->saveToFile(friends); // Implement this to write all friendships in memory
  friends.close();

  // --- Save Messages ---
  // For each student, write messages_<matricNo>.txt from mm
  current = sm->getHead(); // Corrected redeclaration
  while (current) {
    string filename = "messages_" + current->data->getMatricNo() + ".txt";
    ofstream msgfile(filename);
    MessageNode* msg_head = mm->getMessages(current->data->getMatricNo()); // Get the head of the message queue
    MessageNode* msg_curr = msg_head;
    while(msg_curr) { // Iterate through the linked list
      msgfile << msg_curr->sender << "|" << msg_curr->text << std::endl;
      msg_curr = msg_curr->next;
    }
    msgfile.close();
    current = current->next;
  }
}

In [ ]:
%%writefile CLI.h
#ifndef CLI_H
#define CLI_H

// Include necessary modules for managing students, friends, messages, and files
#include "Student.h"
#include "Friends.h"
#include "Message.h"
#include <string>
using namespace std;

// --- CLI (Command Line Interface) Class ---
// Responsible for handling all user interactions through a text-based menu system.

class CLI {
  private:
    string currentUserId;         // Stores the currently logged-in user's ID (eg. matricNo.)

  public:
    // Constructor
    CLI();

    // Setter
    void setCurrent(string);

    // --- Helper Methods ---
    // Email and password validation methods
    bool isUpper(char ch);                      // Checks for uppercase character
    bool isLower(char ch);                      // Checks for lowercase character
    bool isDigit(char ch);                      // Checks for digit character
    bool isValidUSMEmail(const string& email);  // Validates if the email is in proper USM format
    bool isValidPassword(const string& pw);     // Validates password strength

    // Converts a string to Title Case
    string toTitleCase(const string& input);

    // --- Menu Display Methods ---
    void displayLoginMenu();                          // Shown at startup
    void displayMainMenu();                           // Main navigation after login
    void viewUserProfiles(const Student_manager&);    // View all student profiles
    void displayStudentMenu();                        // Menu for student profile operations
    void displayFriendMenu(Student*, FriendsManager*&, const Student_manager*&);     // Friend management
    void displayInboxAndThread(Student*, MessageManager*&, const FriendsManager*&, const Student_manager*&); // Messaging UI

    // --- Authentication and Registration ---
    bool login();   // Handles user login logic
    void signup();  // Handles user registration logic
    void profileSetUp(const string& email, const string& password);  // Sets up user profile after signup

    // Starts the CLI application
    void run();
};

#endif // CLI_H

In [ ]:
%%writefile CLI.cpp
#include "CLI.h"
#include <iostream>

// Constructor: initializes pointers to manager classes using the provided FileManager
CLI::CLI() {
  currentUserId = "";  // initially no user is logged in
}

void CLI::setCurrent(string id) {currentUserId = id;}

// ==== Helper Methods ====
bool CLI::isUpper(char ch) { return ch >= 'A' && ch <= 'Z'; } // Checks if a character is uppercase
bool CLI::isLower(char ch) { return ch >= 'a' && ch <= 'z'; } // Checks if a character is lowercase
bool CLI::isDigit(char ch) { return ch >= '0' && ch <= '9'; } // Checks if a character is a digit

// Validates that the email is a USM student email
bool CLI::isValidUSMEmail(const string& email) {
  string domain = "@student.usm.my";
  if (email.length() > domain.length() + 2) {                           // ab@student.usm.my (length = 18) → passes.
    return email.substr(email.length() - domain.length()) == domain;    // a@student.usm.my (length = 17) → fails (not enough characters before the domain).
  }                                                                     // substr to start from the @ to the end to check domain
  return false;
}

// Validates password strength: must contain upper, lower, and digit
bool CLI::isValidPassword(const string& pw) {
  bool hasUpper = false, hasLower = false, hasDigit = false;
  for (char ch : pw) {  // Range based for loop is safer (no type casting or index overflow).
    if (isUpper(ch))
      hasUpper = true;
    else if (isLower(ch))
      hasLower = true;
    else if (isDigit(ch))
      hasDigit = true;
  }
  return hasUpper && hasLower && hasDigit;
}

// Converts an input string to title case
// The first letter of each word is capitalized, the rest are lowercase.
string CLI::toTitleCase(const string& input) {
  string result = input;     // Make a copy of the input to modify
  bool newWord = true;       // Flag to check if current character starts a new word

  for (int i = 0; i < (int)result.length(); ++i) {
    char& ch = result[i];  // Reference to the current character

    // Check for word boundaries (space or hyphen)
    if (ch == ' ' || ch == '-') {
      newWord = true;    // Next character will start a new word
    } else {
      if (newWord && ch >= 'a' && ch <= 'z') {
          ch -= 32;      // Convert first letter of the word to uppercase (ASCII)
      } else if (!newWord && ch >= 'A' && ch <= 'Z') {
          ch += 32;      // Convert non-first uppercase letters to lowercase (ASCII)
      }
      newWord = false;   // Subsequent characters are not the start of a word
    }
  }
  return result;  // Return the modified string
}

// ==== Menu Display Methods ====

// Display main login page menu
void CLI::displayLoginMenu() {
  cout << "+--------------------------------------------+" << endl;
  cout << "|     Student Networking CLI Application     |" << endl;
  cout << "+--------------------------------------------+" << endl;
  cout << "| [1] Login                                  |" << endl;
  cout << "| [2] Register                               |" << endl;
  cout << "| [3] Exit                                   |" << endl;
  cout << "+--------------------------------------------+" << endl;
}

// Display main application menu after login
void CLI::displayMainMenu() {
  cout << "+---------------------------------------+" << endl;
  cout << "|               Main Menu               |" << endl;
  cout << "+---------------------------------------+" << endl;
  cout << "| [1] View All Student Profiles         |" << endl;
  cout << "| [2] View Personal Profile             |" << endl;
  cout << "| [3] Friends                           |" << endl;
  cout << "| [4] View Personal Inbox               |" << endl;
  cout << "| [5] Undo Actions                      |" << endl;
  cout << "| [6] View Group Chats                  |" << endl;
  cout << "| [7] Logout                            |" << endl;
  cout << "+---------------------------------------+" << endl;
}

void CLI::viewUserProfiles(const Student_manager& sm) {
  int studMenuOpt = 0;
  sm->printAscList();

  do {
    displayStudentMenu();
    cout << "Enter your choice: ";
    cin >> studMenuOpt;
    cin.ignore(10000, '\n');

    switch (studMenuOpt) {
      case 1: // Sorts list based on matric no and print list in ascending
        sm->sortList(MATRIC);
        sm->printAscList();
        break;

      case 2: // Sorts list based on fullname and print list in ascending
        sm->sortList(NAME);
        sm->printAscList();
        break;

      case 3: // Sorts list based on course and print list in ascending
        sm->sortList(COURSE);
        sm->printAscList();
        break;

      case 4: // Prints list ascending based on the previous sort (default sort: matric no)
        sm->printAscList();
        break;

      case 5: // Prints list descending based on the previous sort (default sort: matric no)
        sm->printDescList();
        break;

      case 6: // Return to main menu
        sm->sortList(MATRIC); // Reset to default sort
        cout << "<Returning to Main Menu...>\n";
        break;

      default:
        cout << "<Invalid choice. Please try again.>\n";
    }
  } while (studMenuOpt != 6);
}

// Display submenu for student sorting options
void CLI::displayStudentMenu() {
  cout << "\n+------------------------------------+" << endl;
  cout << "|           Student Menu             |" << endl;
  cout << "+------------------------------------+" << endl;
  cout << "| [1] Sort by Matric Number          |" << endl;
  cout << "| [2] Sort by Name                   |" << endl;
  cout << "| [3] Sort by Course                 |" << endl;
  cout << "| [4] Print Ascending List           |" << endl;
  cout << "| [5] Print Descending List          |" << endl;
  cout << "| [6] Back                           |" << endl;
  cout << "+------------------------------------+" << endl;
}

// Friend management menu
void CLI::displayFriendMenu(Student* currentUser, FriendsManager*& fm, const Student_manager*& sm) {

  // If no user is logged in (currentUser is null), exit immediately.
  if (!currentUser) return;

  // Get the specific friend list for the current user from the FriendsManager.
  FriendList* fl = fm->findOrCreateList(currentUser->getMatricNo());

  // variables to store the user's menu choice and an matricNo. for friend operations.
  int friendOpt;
  string id;

  do {
    cout << "\n+------------------------------------+" << endl;
    cout << "|             Friends Menu           |" << endl;
    cout << "+------------------------------------+" << endl;
    cout << "| [1] View Friend List               |" << endl;
    cout << "| [2] Add a Friend                   |" << endl;
    cout << "| [3] Remove a Friend                |" << endl;
    cout << "| [4] Back                           |" << endl;
    cout << "+------------------------------------+" << endl;
    cout << "Enter choice: ";
    cin >> friendOpt;
    cin.ignore(10000, '\n');  // clear input buffer

      // Menu-driven switch statement to manage a user's friend list
      switch (friendOpt) {
        case 1: // Print the user's friend list

          if (fl)
            fl->printList(*sm); // Print friend list if it exists
          else {
            cout << "+--------------------------------------------------+" << endl;
            cout << "|            You have no friends added.            |" << endl;
            cout << "+--------------------------------------------------+" << endl;
          }

          break;

        case 2: // Option 2: Add a new friend
          cout << "+--------------------------------------------------+" << endl;
          cout << "|                     Add a friend                 |" << endl;
          cout << "+--------------------------------------------------+" << endl;
          cout << "Enter Matric No. of your friend: ";
          getline(cin, id); // Get friend's matric number input

          if (id == currentUser->getMatricNo())
            cout << "<You cannot add yourself.>\n";                // Prevent user from adding themselves
          else {
            Node* studentNode = sm->searchData(MATRIC, id);        // Search for the student by matric no.
            if (!studentNode)
                cout << "<Student not found.>\n";                  // If student doesn't exist
            else if (fl && fl->isFriend(id))
                cout << "<Already a friend.>\n";                   // Prevent duplicate friendships
            else {
                fm->addFriendship(currentUser->getMatricNo(), id); // Add friendship relation
                cout << "<Friend added successfully.>\n";
            }
          }
          break;

        case 3: // Option 3: Remove an existing friend
          cout << "+--------------------------------------------------+" << endl;
          cout << "|                  Remove a friend                 |" << endl;
          cout << "+--------------------------------------------------+" << endl;
          cout << "Enter Matric No. of friend to remove: ";
          getline(cin, id); // Get matric number of friend to remove

          if (!fl || !fl->isFriend(id))
            cout << "<This student is not in your friend list.>\n";     // Check if they're actually a friend
          else {
            fl->removeFriend(id);                                       // Remove friend from current user's list
            FriendList* friendUserFL = fm->getFriendList(id);           // Get friend's list
            if (friendUserFL)
              friendUserFL->removeFriend(currentUser->getMatricNo()); // Remove user from friend's list
            cout << "<Friend removed successfully.>\n";
          }
          break;

        case 4: // Option 4: Exit the friend management menu
          cout << "Returning to Main Menu...\n";
          break;

        default:
          // Handle invalid option input
          cout << "Invalid option. Try again.\n";
      }

  } while (friendOpt != 4);
}

// Handles inbox actions: viewing threads, sending messages or images
void CLI::displayInboxAndThread(Student* currentUser, MessageManager*& mm,
                                 const FriendsManager*& fm, const Student_manager*& sm) {
  if (!currentUser) return;

  string myId = currentUser->getMatricNo();
  int choice;

  do {
    // Display inbox menu
    cout << "\n+-----------------------------+\n";
    cout << "| [1] View Inbox & Thread     |\n";
    cout << "| [2] Send Message            |\n";
    cout << "| [3] Send Image              |\n";
    cout << "| [4] Back                    |\n";
    cout << "+-----------------------------+\n";
    cout << "Enter choice: ";
    cin >> choice;
    cin.ignore(10000, '\n');

    if (choice == 1) {
      // --- View chat thread ---
      cout << "Enter friend's matric number: ";
      string friendId;
      getline(cin, friendId);
      mm->printThread(myId, friendId, sm);

    } else if (choice == 2) {
      // --- Send message ---
      FriendList* fl = fm->getFriendList(myId);
      if (!fl || !fl->getHead()) {
        cout << "<You have no friends to message.>\n";
        continue;
      }

      cout << "Your friends:\n";
      fl->printList(*sm);
      cout << "Enter friend's matric number: ";
      string toId;
      getline(cin, toId);

      // Validate if friend exists
      if (!fl->isFriend(toId)) {
        cout << "<You can only send messages to your friends.>\n";
        continue;
      }

      Node* receiverNode = sm->searchData(MATRIC, toId);
      if (!receiverNode) {
        cout << "<Student not found.>\n";
        continue;
      }

      cout << "Enter your message: ";
      string msgText;
      getline(cin, msgText);

      if (msgText.empty()) {
        cout << "<No message entered.>\n";
        continue;
      }

      mm->addMessage(toId, myId, msgText);
      cout << "<Message sent!>\n";

    } else if (choice == 3) {
      // --- Send image ---
      FriendList* fl = fm->getFriendList(myId);
      if (!fl || !fl->getHead()) {
        cout << "You have no friends to message.\n";
        continue;
      }

      cout << "Your friends:\n";
      fl->printList(*sm);
      cout << "Enter friend's matric number: ";
      string toId;
      getline(cin, toId);

      if (!fl->isFriend(toId)) {
        cout << "You can only send images to your friends.\n";
        continue;
      }

      Node* receiverNode = sm->searchData(MATRIC, toId);
      if (!receiverNode) {
        cout << "Student not found.\n";
        continue;
      }

      cout << "Enter image file name (e.g., photo.jpg): ";
      string imgFile;
      getline(cin, imgFile);

      if (imgFile.empty()) {
        cout << "No file name entered.\n";
        continue;
      }

      mm->addMessage(toId, myId, "[IMAGE]" + imgFile);
      cout << "Image sent as message!\n";
    }

  } while (choice != 4); // Exit inbox
}

// ==== Authentication and Registration ====

// Handles user login process
bool CLI::login(const Student_manager*& sm) {
  string email, password;

  cout << "+-----------------------------------+" << endl;
  cout << "|               Login               |" << endl;
  cout << "+-----------------------------------+" << endl;

  // Prompt user to enter email and password
  cout << "USM Student Email: ";
  cin >> email;
  cout << "Password: ";
  cin >> password;

  // Search for student record by email
  Node* stud = sm->searchData(EMAIL, email);
  if (!stud) {
    // If no matching student found
    cout << "<Student not found for email: " << email << ">" << endl;
    return false;
  }

  // Check if entered password matches stored password
  if (stud->data->getPassword() == password) {
    // Set the currently logged-in user's ID
    currentUserId = stud->data->getMatricNo();
    cout << "<Login successful!>\n\n";
    return true;
  } else {
      // If password does not match
      cout << "<Incorrect password!>\n";
      return false;
  }
}

// Handles the sign-up process for new users, including email and password validation
void CLI::signup(Student_manager*& sm) {
  string email, password;
  bool valid = false;

  // Display the sign-up banner and instructions
  cout << "+-------------------------------------+" << endl;
  cout << "|               Sign Up               |" << endl;
  cout << "+-------------------------------------+" << endl;
  cout << "Enter (-1) anytime to return to login page." << endl << endl;

  // --- Email Input and Validation ---
  do {
    cout << "Enter Email: ";
    cin >> email;

    // Allow user to cancel sign-up and return to login
    if (email == "-1") return;

    // Check if email is already registered in the system
    if (sm->searchData(EMAIL, email))
        cout << "Email already exists!\n";

    // Validate that email belongs to USM domain (e.g., ends with @student.usm.my)
    else if (!isValidUSMEmail(email))
        cout << "Invalid email domain! Please use a student email!\n";

    // Limit email length to 30 characters
    else if (email.length() > 30)
        cout << "Email too long!\n";

    // All checks passed, accept the email
    else
        valid = true;

  } while (!valid);  // Repeat until valid email is provided

  valid = false;  // Reset for password validation

  // --- Password Input and Validation ---
  do {
    // Inform user of password requirements
    cout << "** Password must contain uppercase, lowercase and digit\n";
    cout << "Enter Password: ";
    cin >> password;

    // Allow user to cancel sign-up
    if (password == "-1") return;

    // Ensure minimum password length
    if (password.length() < 8)
        cout << "Password too short!\n";

    // Ensure maximum password length
    else if (password.length() > 20)
        cout << "Password too long!\n";

    // Check if password meets complexity requirements
    else if (!isValidPassword(password))
        cout << "Password must contain uppercase, lowercase and digit!\n";

    // All checks passed, accept the password
    else
        valid = true;

  } while (!valid);  // Repeat until valid password is provided

  // Proceed to profile setup with the validated email and password
  profileSetUp(email, password);

  // Inform user of return to login
  cout << "<Returning to login page...>\n";
}

// Handles the process of collecting and validating student's personal profile information
void CLI::profileSetUp(const string& email, const string& password, Student_manager*& sm, FriendsManager*& fm) {
  string matricNo, fullName, courseName;
  bool valid = false;

  // --- Matric Number Validation ---
  do {
      cout << "Enter Matric Number (8-digit): ";
      cin >> matricNo;

      // Check for exact 8-digit length
      if (matricNo.length() != 8)
          cout << "Matric number must be 8 digits!\n";

      // Check for uniqueness in the system
      else if (sm->searchData(MATRIC, matricNo))
          cout << "Matric number already exists!\n";

      // Accept valid matric number
      else
          valid = true;

  } while (!valid);

  cin.ignore();  // Clear newline from input buffer

  valid = false;  // Reset for name validation

  // --- Full Name Input and Validation ---
  do {
      cout << "Enter Full Name: ";
      getline(cin, fullName);

      // Convert name to title case (capitalize first letters)
      fullName = toTitleCase(fullName);

      // Check if name is already used (optional, depends on system rules)
      if (sm->searchData(NAME, fullName))
          cout << "Name already exists!\n";
      else
          valid = true;

  } while (!valid);

  int choice;

  // --- Course Selection ---
  do {
      cout << "+-------------------------------------+" << endl;
      cout << "|           Select your course        |" << endl;
      cout << "+-------------------------------------+" << endl;
      cout << "| [1] Computing Infrastructure        |" << endl;
      cout << "| [2] Intelligent Computing           |" << endl;
      cout << "| [3] Software Engineering            |" << endl;
      cout << "+-------------------------------------+" << endl;
      cout << "Enter your choice: ";
      cin >> choice;

      // Assign course name based on user's choice
      if (choice == 1)
          courseName = "Computing Infrastructure";
      else if (choice == 2)
          courseName = "Intelligent Computing";
      else if (choice == 3)
          courseName = "Software Engineering";
      else
          cout << "<Invalid choice. Please try again.>\n";

  } while (choice < 1 || choice > 3);

  // --- Display Collected Info for Confirmation ---
  cout << "Matric No: " << matricNo << endl;
  cout << "Full Name: " << fullName << endl;
  cout << "Course: " << courseName << endl;

  // Ask user to confirm the entered information
  cout << "Confirm data (Y/N): ";
  char confirm;
  cin >> confirm;

  // --- Final Confirmation and Student Creation ---
  if (confirm == 'y' || confirm == 'Y') {
      // Create new student object with all provided details
      Student* stud = new Student(matricNo, fullName, email, password, courseName);

      // Insert the new student into the student manager (likely a sorted list)
      sm->sortedInsert(stud);

      cout << "<New profile sign up successfully!>\n";
      fm->findOrCreateList(matricNo); // Create a new friend list for the new student
  } else {
      cout << "<New profile sign up cancelled.>\n";
  }
}


// ==== Main CLI Loop ====
// Main loop of the CLI — handles login, signup, main menu navigation, and logout
void CLI::run(Student_manager*& sm, FriendsManager*& fm, MessageManager*& mm) {
  int loginOpt = 0;
  bool loginSuccess = false;

  do {
    displayLoginMenu();
    cout << "Enter your choice: ";
    cin >> loginOpt;
    cin.ignore(10000, '\n'); // Clear input buffer

    switch (loginOpt) {
      case 1: // Login selected
        if (login()) {
          loginSuccess = true;
          int mainOpt = 0;

          do {
            displayMainMenu();
            cout << "Enter your choice: ";
            cin >> mainOpt;
            cin.ignore(10000, '\n'); // Clear input buffer

            // Retrieve currently logged-in student
            Node* currentUserNode = sm->searchData(MATRIC, currentUserId);

            switch (mainOpt) {
              case 1: // View all student profiles
                if(currentUserNode)
                  viewUserProfiles(sm);
                else
                  cout << "<Error: Current user profile not found.>\n";
                break;

              case 2: // View personal profile
                if (currentUserNode) {
                  cout << "+---------------------------------------+" << endl;
                  cout << "|            Personal Profile           |" << endl;
                  cout << "+---------------------------------------+" << endl;
                  sm->printHeader();
                  currentUserNode->data->printInfo();
                  sm->printLine();
                } else
                  cout << "<Error: Current user profile not found.>\n";
                break;

              case 3: // Friend list and operations
                if (currentUserNode)
                  displayFriendMenu(currentUserNode->data, fm, sm);
                else
                  cout << "<Error: Current user profile not found for friend list.>\n";
                break;

              case 4: // Inbox and messaging
                if (currentUserNode)
                  displayInboxAndThread(currentUserNode->data, mm, fm, sm);
                else
                  cout << "<Error: Current user profile not found for inbox.>\n";
                break;

              case 5: // Undo feature placeholder
                break;

              case 6: // Group chat feature placeholder
                break;

              case 7: // Logout
                loginSuccess = false;
                cout << "Logging out...\n";
                break;

              default:
                  cout << "Invalid choice. Please try again.\n";
            }
          } while (loginSuccess);
        }
        break;

        case 2: // Sign up
          signup();
          break;

        case 3: // Exit
          cout << "Exiting program...\n";
          break;

        default:
          cout << "Invalid choice. Please try again.\n";
      }
  } while (loginOpt != 3); // Keep running until user chooses to exit
}

In [ ]:
%%writefile main.cpp
#include <iostream>
#include "FileManager.h" // Includes the FileManager class for handling all file I/O
#include "CLI.h"         // Includes the Command Line Interface class

int main() {
  // Create an instance of FileManager to manage students, friends, and messages
  FileManager* fileManager = new FileManager();     // Handles file I/O
  StudentManager* sm = new Student_manager();           // Manages student records
  FriendsManager* fm = new FriendsManager();           // Manages friendships
  MessageManager* mm = new MessageManager();           // Manages messaging

  // Load all saved data from files (students, friends, messages, etc.)
  fileManager.loadAll(sm, fm, mm);

  // Initialize the CLI with a pointer to the FileManager so it can access data
  CLI cli;

  // Start the command line interface loop (user interaction begins here)
  cli.run(sm, fm, mm);

  // After the user exits the CLI, save all updated data back to files
  fileManager.saveAll(sm, fm, mm);

  return 0; // End the program successfully
}
